In [1]:
pip install hmmlearn

     |████████████████████████████████| 368kB 4.4MB/s 


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
from datetime import datetime
import calendar
from hmmlearn import hmm

In [3]:
df_1 = pd.read_csv('/content/drive/My Drive/Mental Health blog dataset/beyondblue_depression.csv')
df_2 = pd.read_csv( "/content/drive/My Drive/Mental Health blog dataset/beyondblue_ptsd-trauma.csv")
df_3 = pd.read_csv('/content/drive/My Drive/Mental Health blog dataset/beyondblue_anxiety.csv')
df_4 = pd.read_csv ('/content/drive/My Drive/Mental Health blog dataset/beyondblue_suicidal-thoughts-and-self-harm.csv')

In [4]:
df_List = [df_1 , df_2, df_3, df_4]  
df_1.columns

Index(['Unnamed: 0', 'title', 'author', 'num posts by author', 'date',
       'num likes', 'post', 'category', 'is_reply'],
      dtype='object')

In [5]:
for data_frame in df_List : 
  data_frame['index_indi'] = pd.Series ([i for i in range (data_frame.shape[0])])

In [6]:
df = pd.concat(df_List, ignore_index=True)

In [7]:
for i in range (df.date.shape[0]) : 
  df.date[i] = datetime.strptime(df.date[i], '%d %B %Y')
df.sort_values(by=['date'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
unique_name = df.author.unique().tolist()
unique_name = np.unique(np.array(unique_name))

In [9]:
post_per_person = []
for k, name in enumerate (unique_name) :
  post_counts = df.loc[df['author'] == name].shape[0]
  post_per_person.append ( [name, int (post_counts)] )

In [10]:
temp = []
for i in range (len(post_per_person)) : 
  temp.append (post_per_person[i][1])
sum (temp) / len(post_per_person)

4.195852041907205

In [11]:
sub_data_names = [] 
for i, freq in enumerate (temp) : 
  if freq >= 5 and freq <= 50 : 
    sub_data_names.append (post_per_person[i][0])


In [12]:
sub_data_set = [] 
for name in sub_data_names : 
  sub_data_set.append (df.loc[df.author == name])

In [13]:
train_cases = []
for i in range(len ( sub_data_set) ) : 
    cat = sub_data_set[i]['category'].to_numpy()
    if 'suicidal-thoughts-and-self-harm' in cat and 'depression' in  cat: 
      temp_1 = sub_data_set[i][sub_data_set[i].category == 'suicidal-thoughts-and-self-harm']
      temp_2 = sub_data_set[i][sub_data_set[i].category == 'depression']
      sub_data_set[i] = pd.concat([temp_1, temp_2])
      sub_data_set[i].sort_values(by=['date'], inplace=True)
      train_cases.append (sub_data_set[i])

In [14]:
len(train_cases)

70

there are 70 unique users who hame posted in depression forum as well as suicide forum. 


In [15]:
LIWC_anxity = pd.read_csv ("/content/drive/My Drive/Mental Health blog dataset/LIWC2015 Results (anxiety (17189 files)).csv")
LIWC_depression = pd.read_csv ( "/content/drive/My Drive/Mental Health blog dataset/LIWC2015 Results (depression (16131 files)).csv")
LIWC_suicide = pd.read_excel ( "/content/drive/My Drive/Mental Health blog dataset/LIWC2015 Results (selfharm (1590 files)).xlsx")
LIWC_PTSD = pd.read_excel ( "/content/drive/My Drive/Mental Health blog dataset/LIWC2015 Results (trauma (4338 files)).xlsx" ) 

LIWC_df = pd.concat([LIWC_anxity, LIWC_depression, LIWC_suicide, LIWC_PTSD])

In [16]:
col = LIWC_df.columns[2:]

In [17]:
LIWC_sequence = []

for case in train_cases : 
  temp = []
  for index, cat in zip ( case.index.to_numpy(), case.category.to_numpy() ): 
    if cat == 'suicidal-thoughts-and-self-harm' : 
      file_name = "selfharm_" + str (index) + "_.txt" 
    elif cat == 'depression' : 
      file_name = "depression_" + str (index) +"_.txt" 
    liwc_vec = LIWC_df[LIWC_df.Filename == file_name][col].to_numpy().flatten()
    if liwc_vec.shape[0] > 0 : 
      temp.append (liwc_vec)
  if len (temp) >= 5: 
    LIWC_sequence.append (temp)  

In [18]:
lengths = []
X = []
for i in range (len(LIWC_sequence)) : 
  lengths.append (len(LIWC_sequence[i]))  
  X += LIWC_sequence[i]

In [19]:
model = hmm.GaussianHMM(2, "full", algorithm='viterbi')
model.fit (X, lengths)

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=2, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

In [20]:
model.predict ([X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7]] ), train_cases[0].category.to_numpy() 

(array([1, 0, 0, 0, 1, 1, 0, 1]),
 array(['depression', 'depression', 'depression', 'depression',
        'suicidal-thoughts-and-self-harm', 'depression', 'depression',
        'depression'], dtype=object))

In [21]:
model.predict ([X[8], X[9], X[10], X[11], X[12]), train_cases[1].category.to_numpy() 

(array([1, 1, 0, 1, 1, 0, 1]),
 array(['suicidal-thoughts-and-self-harm', 'depression', 'depression',
        'suicidal-thoughts-and-self-harm',
        'suicidal-thoughts-and-self-harm'], dtype=object))